# Read OpenAI API key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("secrets.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Call to OpenAI API

**Pros of Using OpenAI Chat API in a Company's Chatbot:**

1. **Advanced Natural Language Processing (NLP):** The API uses state-of-the-art NLP, allowing the chatbot to understand and respond to user queries more effectively, even with complex or ambiguous language.
2. **Versatility:** OpenAI's API can handle a wide variety of topics, making the chatbot adaptable to diverse user needs across multiple domains without extensive retraining.
3. **Scalability:** The API can handle high volumes of requests, enabling businesses to scale their chatbot solutions without worrying about performance degradation.
4. **Continuous Improvement:** OpenAI continually updates and improves its models, so your chatbot can benefit from cutting-edge advancements in AI without needing significant internal resources.
5. **Rapid Deployment:** Implementing the OpenAI Chat API can accelerate the development and deployment of a chatbot, reducing time-to-market compared to building a custom NLP system from scratch.

**Cons of Using OpenAI Chat API in a Company's Chatbot:**

1. **Cost:** Usage of the API incurs ongoing costs based on the volume of requests, which can add up, especially for high-traffic applications.
2. **Limited Customization:** Although versatile, the API may not always meet highly specific business requirements, requiring additional layers of customization that can be complex to implement.
3. **Data Privacy Concerns:** Using a third-party API may raise concerns about data privacy and security, particularly in industries that handle sensitive customer information.
4. **Dependency on External Service:** Relying on an external service like OpenAI means potential downtime, rate-limiting, or API changes that are beyond the company's control.
5. **Ethical Considerations:** AI-generated responses can sometimes produce unexpected or inappropriate content, which may require careful monitoring and mitigation strategies to maintain brand integrity.

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is AI?",
        }
    ],
    model="gpt-3.5-turbo",
)

print(chat_completion.choices[0].message.content)

AI, or artificial intelligence, refers to the capability of a machine or computer system to perform tasks that would typically require human intelligence. This includes tasks such as learning, reasoning, problem-solving, understanding and interpreting natural language, and recognizing patterns in data. AI systems can learn from data, adapt to new inputs, and perform tasks with a level of autonomy. Some common applications of AI include virtual assistants, image recognition, natural language processing, and autonomous vehicles.


# Do I need to write utility classes on my own? - Utilise LangChain

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
response = model.invoke([HumanMessage(content="What is AI?")])

print(response.content)

AI stands for artificial intelligence, which refers to the simulation of human intelligence processes by machines, especially computer systems. AI involves the development of algorithms and software that enable computers to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI technologies include machine learning, natural language processing, computer vision, and robotics.


In [4]:
response = model.invoke([HumanMessage(content="It is really interesting what you have written. Can you say more about it?")])
print(response.content)

Of course! I can provide more information or elaborate on the topic you're interested in. Just let me know what specifically you would like to know more about.


As you can see - the model by itself doesn't remember the history of messages. We need to pass all messages to get the desired answers.

In [5]:
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="What is AI?"),
        AIMessage(
            content="AI, or artificial intelligence, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI technology aims to mimic human cognitive functions and improve efficiency and accuracy in tasks that were previously only achievable by humans."
        ),
        HumanMessage(content="It is really interesting what you have written. Can you say more about it?"),
    ]
)

print(response.content)

Of course! Artificial intelligence can be categorized into two main types: narrow AI and general AI. Narrow AI, also known as weak AI, is designed to perform specific tasks and is limited to a narrow scope of functions. Examples of narrow AI include virtual assistants like Siri and Alexa, recommendation algorithms on streaming services, and facial recognition technology.

On the other hand, general AI, also known as strong AI, refers to a system that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks similar to human intelligence. General AI is still a theoretical concept and is not yet fully realized in practice.

AI technologies such as machine learning, deep learning, natural language processing, and computer vision play a crucial role in the development of artificial intelligence systems. These technologies enable computers to analyze large amounts of data, recognize patterns, and make decisions without human intervention.

AI has applicati

# Message History

In [6]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config = {"configurable": {"session_id": "session_1"}}

response = chain_with_history.invoke(
    [HumanMessage(content="What is AI?")],
    config=config,
)

print(response.content)

AI stands for artificial intelligence, which refers to the simulation of human intelligence processes by machines, especially computer systems. This includes learning, reasoning, problem-solving, perception, language understanding, and decision-making. AI technologies are used in various applications such as speech recognition, image processing, natural language processing, robotics, and autonomous vehicles.


In [8]:
response = chain_with_history.invoke(
    [HumanMessage(content="It is really interesting what you have written. Can you say more about it?")],
    config=config,
)

print(response.content)

Sure! Artificial intelligence is a rapidly evolving field that aims to create intelligent machines that can perform tasks that typically require human intelligence. AI systems are designed to learn from data, adapt to new information, and make decisions based on patterns and trends in the data. There are different types of AI, including narrow AI (or weak AI) which is designed for specific tasks, and general AI (or strong AI) which aims to replicate human-level intelligence across a wide range of tasks.

Some common techniques used in AI include machine learning, deep learning, neural networks, natural language processing, computer vision, and reinforcement learning. These techniques enable AI systems to analyze and interpret data, recognize patterns, and make predictions or decisions based on the information available to them.

AI is being used in a wide range of industries and applications, including healthcare, finance, transportation, marketing, customer service, and many others. I

After a session_id change, another user will not have access to the previous conversation history.

In [9]:
config = {"configurable": {"session_id": "session_2"}}

response = chain_with_history.invoke(
    [HumanMessage(content="What was I asking about earlier?")],
    config=config,
)

print(response.content)

I'm sorry, I cannot remember what you were asking about earlier as I do not have the capability to retain information from previous interactions. Could you please provide more context or details so I can assist you further?


## Different types of memories

- all messages (presented earlier)
- trim messages
- summarize conversation

### Trim messages

In [10]:
from langchain_core.runnables import RunnablePassthrough


# Trim to four messages
def trim_messages(chain_input):
    current_store = store[config["configurable"]["session_id"]]
    if len(current_store.messages) <= 4:
        return False

    current_store.clear()

    for message in current_store.messages[-4:]:
        current_store.add_message(message)

    return True


chain_with_trimming = RunnablePassthrough.assign(messages_trimmed=trim_messages) | chain_with_history

response = chain_with_trimming.invoke(
    {"input": "My name is Bob."},
    config=config,
)
print(f"1: {response.content}")

response = chain_with_trimming.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"2: {response.content}")

response = chain_with_trimming.invoke(
    {"input": "What is my name?"},
    config=config,
)
print(f"3: {response.content}")

1: Nice to meet you, Bob! How can I assist you today?
2: Your name is Bob.
3: I'm sorry, I do not know your name as I am an AI assistant and do not have that information.


### Summarize conversation

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def summarize_messages(chain_input):
    current_store = store[config["configurable"]["session_id"]]
    if len(current_store.messages) == 0:
        return False

    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="chat_history"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )

    summarization_chain = summarization_prompt | model
    summary_message = summarization_chain.invoke({"chat_history": current_store.messages})

    current_store.clear()
    current_store.add_message(summary_message)

    return True


chain_with_summarization = RunnablePassthrough.assign(messages_summarized=summarize_messages) | chain_with_history

response = chain_with_summarization.invoke(
    {"input": "My name is Bob."},
    config=config,
)
print(f"1: {response.content}")

response = chain_with_summarization.invoke(
    {"input": "How did I introduce myself?"},
    config=config,
)
print(f"2: {response.content}")

response = chain_with_summarization.invoke(
    {"input": "How did I introduce myself?"},
    config=config,
)
print(f"3: {response.content}")
print(store[config["configurable"]["session_id"]].messages)

1: Nice to meet you, Bob! How can I assist you today?
2: You introduced yourself by typing your name, Bob, in your message.
3: You introduced yourself by typing your name, "Bob."
[AIMessage(content='In the chat conversation, Bob introduced himself by typing his name, and the AI assistant acknowledged his name. The AI assistant expressed readiness to assist Bob with any queries or tasks he may have.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 94, 'total_tokens': 132}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ba73d24e-ff3c-4e30-8c45-1b47bc0c9094-0', usage_metadata={'input_tokens': 94, 'output_tokens': 38, 'total_tokens': 132}), HumanMessage(content='How did I introduce myself?'), AIMessage(content='You introduced yourself by typing your name, "Bob."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 

# Prompting techniques

- Use delimiters to clearly indicate distinct parts of the input
- Ask for a structured output
- Ask the model to check whether conditions are satisfied
- "Few-shot" prompting
- Specify the steps required to complete a task

## Use delimiters to clearly indicate distinct parts of the input

In [12]:
text = f"""
You should express what you want a model to do by
providing instructions that are as clear and
specific as you can possibly make them.
This will guide the model towards the desired output,
and reduce the chances of receiving irrelevant
or incorrect responses. Don't confuse writing a
clear prompt with writing a short prompt.
In many cases, longer prompts provide more clarity
and context for the model, which can lead to
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

It is important to provide clear and specific instructions to guide a model towards the desired output, even if that means writing a longer prompt for more clarity and context.


## Ask for a structured output

In [13]:
prompt = """
Generate a list of three made-up book titles along with their number of pages (int), release date (yyyy-mm-dd) and if cover is hard or not (boolean).
Provide them in a yaml structured output format where title name is a key with three elemnts: number of pages, release date and if cover type.
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

```yaml
- Title1:
    pages: 320
    release_date: 2023-07-15
    hard_cover: true
- Title2:
    pages: 250
    release_date: 2024-02-29
    hard_cover: false
- Title3:
    pages: 400
    release_date: 2022-09-10
    hard_cover: true
```


### Task - try to rewrite this prompt to get the results in JSON format

In [14]:
prompt = """
Place for your prompt
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

What is your favorite childhood memory?


## Ask the model to check whether conditions are satisfied

In [15]:
text_1 = f"""
Preheat the oven to 350°F (175°C).
Mix 1 cup of softened butter, 1 cup of sugar, 2 cups of flour, and 1 tsp vanilla extract until combined.
Scoop spoonfuls onto a baking sheet and bake for 10-12 minutes, until golden brown.
Let cool, and enjoy!
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)


Step 1 - Preheat the oven to 350°F (175°C).
Step 2 - Mix 1 cup of softened butter, 1 cup of sugar, 2 cups of flour, and 1 tsp vanilla extract until combined.
Step 3 - Scoop spoonfuls onto a baking sheet and bake for 10-12 minutes, until golden brown.
Step 4 - Let cool, and enjoy!


In [16]:
text_2 = f"""
A short story is a piece of prose fiction. It can typically be read
in a single sitting and focuses on a self-containedincident or series
of linked incidents, with the intent of evoking a single effect or mood.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions,
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

No steps provided.


## "Few-shot" prompting

In [17]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest
valley flows from a modest spring; the
grandest symphony originates from a single note;
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about perseverance.
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

<grandparent>: Perseverance is like the sturdy oak tree that weathers every storm, standing tall and strong despite the challenges it faces. It is the unwavering determination to keep moving forward, even when the path is difficult and the end is not in sight. Just as the oak tree grows stronger with each passing year, so too does our resolve with every obstacle we overcome.


## Specify the steps required to complete a task

In [18]:
text = """
Preheat the oven to 350°F (175°C).
Mix 1 cup of softened butter, 1 cup of sugar, 2 cups of flour, and 1 tsp vanilla extract until combined.
Scoop spoonfuls onto a baking sheet and bake for 10-12 minutes, until golden brown.
Let cool, and enjoy!
"""

prompt = f"""
I will give you text delimited with ###.
Perform the following actions:

1 - If it contains the instruction, rewrite them in the following format:

A - <instruction 1>
B - <instruction 2>
C - <instruction 3>

2 - Translate each instruction to one of the following languages: spanish, french, german, polish, norwegian.
3 - Output a list of objects in a json format. Each object in a list is for one translated sentence and contains the following keys:
original_sentence, translated_sentence, translation_language. There should be as many objects as instructions in the previous step.
4 - If it does not contin instructions output json object with a key \"error" and value for it which will be an error message

###{text}###
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

{
  "objects": [
    {
      "original_sentence": "Preheat the oven to 350°F (175°C).",
      "translated_sentence": "Vorheizen des Ofens auf 350°F (175°C).",
      "translation_language": "german"
    },
    {
      "original_sentence": "Mix 1 cup of softened butter, 1 cup of sugar, 2 cups of flour, and 1 tsp vanilla extract until combined.",
      "translated_sentence": "Mezclar 1 taza de mantequilla ablandada, 1 taza de azúcar, 2 tazas de harina y 1 cdta de extracto de vainilla hasta que se combine.",
      "translation_language": "spanish"
    },
    {
      "original_sentence": "Scoop spoonfuls onto a baking sheet and bake for 10-12 minutes, until golden brown.",
      "translated_sentence": "Déposer des cuillerées sur une plaque de cuisson et cuire au four pendant 10 à 12 minutes, jusqu'à ce qu'elles soient dorées.",
      "translation_language": "french"
    },
    {
      "original_sentence": "Let cool, and enjoy!",
      "translated_sentence": "Lassen Sie abkühlen und genießen

### Task - Write a prompt that generates three different made-up movies and their descriptions. Each movie name and description should be in a different style (poem, pirate-like, William Shakespeare). The output should be an HTML table. Use all of the aforementioned techniques.

In [19]:
prompt = """
Place for your prompt.
"""

response = model.invoke([HumanMessage(content=prompt)])
print(response.content)

Write a story about a character who discovers a hidden underground world beneath their town and must navigate through it to uncover its secrets.


# Prompt templates

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant and a pirate. Answer all questions to the best of your ability, but remember to use pirate-like english.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model
pirate_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = pirate_chain_with_history.invoke({"input": "What is AI?"}, config=config)
print(response.content)

Arr matey, AI be short for "Artificial Intelligence." 'Tis a technology where machines be programmed to think and learn like humans, performin' tasks such as answerin' questions, solvin' problems, and helpin' sailors like ye with all sorts o' matters.


# Build RAG - add vector storage
## Initialize vector storage

In [21]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=OPENAI_API_KEY)

vector_store = Chroma(
    collection_name="shrek_collection", embedding_function=embeddings, persist_directory="./chroma_langchain_db"
)

## Add documents

In [22]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./docs/shrek-script.pdf")
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(document)

vector_store.add_documents(chunked_documents)

['37a91407-ec1a-444c-97f5-6f06b572aa98',
 '5dc04ff3-7824-4c14-88f2-01679ae253b9',
 '6dafdc99-ab24-4972-97ea-441a51bcce3d',
 '09479d00-b09d-48d9-a144-c3281be5268a',
 '9dda638d-9e88-4752-a7ab-f055b2b11380',
 '1a375d1d-be97-4daa-b05a-fa43fbab10e1',
 '524f3e98-262e-448d-bd1a-8ab1522b9e67',
 '00da17c5-ee42-4343-a2ad-5ce671ddefa5',
 '9c294cb2-296f-47ad-9de3-a8b162bdfb79',
 '140ea5ef-013e-46e1-b034-9fecdde0f25c',
 'beffb535-acf7-400f-8b60-b1d1179fb725',
 'a0b73463-4054-4aa3-9662-62b73c1ba24a',
 'ffab9b3c-5220-4af7-bea6-69d9cc562ba6',
 '654c302d-d9c7-4490-86db-262745fe169d',
 'f1f3f345-25e6-4b31-b672-8fcc67976d0e',
 '62130038-2c3a-4c58-b96b-591758a7593a',
 'af56ad46-e240-4e79-b7a5-035356b6a297',
 '53d6eae3-322c-42dd-ae97-75f29256c436',
 '132abe31-c8a1-4342-a8a5-e62cb51735bd',
 'f5b360b1-b80d-436a-97ea-da5f31c65b8b',
 'd82597df-c16f-4911-87c5-d1c9aafeed1b',
 'f3545741-1bbd-42a5-a6ad-38408a4b602b',
 'd13c13ec-1004-40ba-803f-d1e705bea7ff',
 '89acd970-ed1f-4512-9ec4-0cc83b17686c',
 '99d92c8d-f819-

## Ask and wait for answears :)

In [29]:
config = {"configurable": {"session_id": "rag_session_1"}}

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant and a literature specialist. Answer all questions to the best of your ability.
            During answearing use this context from documents: {context}. If you do not know the answear - do not provide it.
            Answear as short as possible, preferably in one sentence""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | model

rag_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


retriever = vector_store.as_retriever()
rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | rag_chain_with_history
)

response = rag_chain.invoke("What was the name of the princess?", config=config)
print(response.content)

Princess Fiona.


In [24]:
vector_store = Chroma(
    collection_name="shrek_collection_small", embedding_function=embeddings, persist_directory="./chroma_langchain_db"
)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunked_documents = text_splitter.split_documents(document)

vector_store.add_documents(chunked_documents)

config = {"configurable": {"session_id": "rag_session_2"}}

retriever = vector_store.as_retriever()
rag_chain = {
    "context": retriever | format_docs,
    "input": RunnablePassthrough(),
} | rag_chain_with_history

response = rag_chain.invoke("What was the name of the princess?", config=config)
print(response.content)

The name of the princess in the context is Princess Fiona.


# Put it all together - Demo with Frontend

In [30]:
import panel as pn

pn.extension()


async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    message = ""
    for response in rag_chain.stream(contents, config=config):
        message += response.content
        yield message


chat_interface = pn.chat.ChatInterface(callback=callback, callback_user="Shrek director")
chat_interface.servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function callback a..., callback_user='Shrek director', show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])

# Additional task - use HuggingFace model

In [26]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from huggingface_hub import login

HF_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

login(token=HF_TOKEN)

llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        repetition_penalty=1.03,
        temperature=0.1
    ),
)

hg_model = ChatHuggingFace(llm=llm)
chain = prompt | hg_model
pirate_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
rag_chain = {
    "context": retriever | format_docs,
    "input": RunnablePassthrough(),
} | pirate_chain_with_history

response = rag_chain.invoke("How old was pirate school?", config=config)
print(response.content)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/wiktorsmura/.cache/huggingface/token
Login successful
<|system|>
You are a helpful assistant and a literature specialist. Answer all questions to the best of your ability. During answearing use this context from documents: LITTLE BEAR
(crying) This cage is too small.
DONKEY
Please, don't turn me in. I'll 
never be stubborn again. I can 
change. Please! Give me another 
chance!
OLD WOMAN
Oh, shut up. (jerks his rope)
DONKEY
Oh!
HEAD GUARD
Next! What have you got?
GIPETTO
This little wooden puppet.
PINOCCHIO
I'm not a puppet. I'm a real boy. 
(his nose grows)
HEAD GUARD
Five shillings for the possessed 
toy. Take it away.
PINOCCHIO
Father, please! Don't let them do 
this! Help me!

"Wanted. Fai